# DuckDB Connection

In [33]:
!ls -lah

total 1.6G
drwxr-xr-x 1 root root 4.0K Jul 29 01:23 .
drwxr-xr-x 1 root root 4.0K Jul 29 00:50 ..
drwxr-xr-x 4 root root 4.0K Jul 25 13:20 .config
-rw-r--r-- 1 root root 695M Jul 26 03:24 lit_panel_aggr_v3.duckdb
-rw-r--r-- 1 root root 883M Jul 18 22:21 lit_reportingowner.duckdb
drwxr-xr-x 1 root root 4.0K Jul 25 13:21 sample_data


In [14]:
!curl -IL https://3253-machine-learning.s3.ca-central-1.amazonaws.com/lit_panel_aggr_v3.duckdb

HTTP/1.1 200 OK
x-amz-id-2: i72AS7AkhHXNpmtswerILqSSrZYMKtDoFE2mWqqShIvgAGxUlUi9osdvCOfhpGpaiG5hyytqDLeRl0a0r0ekuapewpQ9uxt/DXdMVCthP1w=
x-amz-request-id: G4X9SCG7KDJ1KBFX
Date: Mon, 29 Jul 2024 00:55:54 GMT
Last-Modified: Fri, 26 Jul 2024 03:24:09 GMT
ETag: "3f08ffe0a0433c624fb7b08a95ab7ff8-87"
x-amz-server-side-encryption: AES256
Accept-Ranges: bytes
Content-Type: binary/octet-stream
Server: AmazonS3
Content-Length: 728248320



In [15]:
!time wget --no-clobber https://3253-machine-learning.s3.ca-central-1.amazonaws.com/lit_panel_aggr_v3.duckdb

File ‘lit_panel_aggr_v3.duckdb’ already there; not retrieving.


real	0m0.030s
user	0m0.000s
sys	0m0.005s


In [16]:
import duckdb

conn = duckdb.connect('lit_panel_aggr_v3.duckdb')

# Schema Exploration

In [17]:
conn.sql('DESCRIBE lit_panel_aggr_v3').show()
conn.sql('SELECT count(*) FROM lit_panel_aggr_v3').show()

┌─────────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│             column_name             │ column_type │  null   │   key   │ default │  extra  │
│               varchar               │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ URL                                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ acceptanceDatetime                  │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ accessionNumber                     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ type                                │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ period                              │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ issuerCIK                           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ documentType                        │ VARCHAR     │ YES   

In [18]:
conn.sql('SELECT * FROM lit_panel_aggr_v3 LIMIT 1').df().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 27 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   URL                                  1 non-null      object 
 1   acceptanceDatetime                   1 non-null      int64  
 2   accessionNumber                      1 non-null      object 
 3   type                                 1 non-null      object 
 4   period                               1 non-null      int64  
 5   issuerCIK                            1 non-null      object 
 6   documentType                         1 non-null      object 
 7   periodOfReport                       1 non-null      object 
 8   notSubjectToSection16                0 non-null      object 
 9   issuerTradingSymbol                  1 non-null      object 
 10  m_isDirector                         1 non-null      int64  
 11  m_isOfficer                         

In [32]:
conn.sql("""SELECT period, transactionCode, _total_transaction_shares, _total_transaction_amount,
              _minSharesOwnedFollowingTransaction as _minSOFT, _maxSharesOwnedFollowingTransaction as _maxSOFT,
              _count
            FROM lit_panel_aggr_v3
            WHERE issuerTradingSymbol = 'AAPL' AND transactionType = 'nonDerivativeTransaction'
            ORDER BY period DESC""").show()

┌──────────┬─────────────────┬───────────────────────────┬───────────────────────────┬──────────┬──────────┬────────┐
│  period  │ transactionCode │ _total_transaction_shares │ _total_transaction_amount │ _minSOFT │ _maxSOFT │ _count │
│  int64   │     varchar     │          int128           │          double           │  int32   │  int32   │ int64  │
├──────────┼─────────────────┼───────────────────────────┼───────────────────────────┼──────────┼──────────┼────────┤
│ 20240530 │ S               │                     75000 │        14368500.000000002 │  4359576 │  4359576 │      1 │
│ 20240515 │ S               │                      4999 │         951784.6050000001 │    29278 │    29278 │      1 │
│ 20240510 │ G               │                      1850 │                       0.0 │   291543 │   291543 │      1 │
│ 20240415 │ M               │                      8119 │                      NULL │    37397 │    37397 │      1 │
│ 20240415 │ F               │                      3120

# Pola.rs

In [19]:
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 67.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [20]:
import sklearn
display(sklearn.__version__)

'1.5.1'

In [21]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    [
        (
            'numerical',
            StandardScaler(),
            ['acceptanceDatetime', 'period',
             '_total_transaction_shares', '_total_transaction_amount',
             '_minSharesOwnedFollowingTransaction', '_maxSharesOwnedFollowingTransaction',
             '_count']
        ),
        (
            'categorical',
            OneHotEncoder(sparse_output=False),
            ['type', 'documentType', 'notSubjectToSection16', 'm_isDirector', 'm_isOfficer', 'm_isTenPercentOwner', 'm_isOther',
             'transactionType', 'transactionFormType', 'transactionCode', 'equitySwapInvolved', 'directOrIndirectOwnership', 'natureOfOwnership']
        )
    ],
    verbose_feature_names_out=False
)

transformer.set_output(transform='polars')



ColumnTransformer(transformers=[('numerical', StandardScaler(),
                                 ['acceptanceDatetime', 'period',
                                  '_total_transaction_shares',
                                  '_total_transaction_amount',
                                  '_minSharesOwnedFollowingTransaction',
                                  '_maxSharesOwnedFollowingTransaction',
                                  '_count']),
                                ('categorical',
                                 OneHotEncoder(sparse_output=False),
                                 ['type', 'documentType',
                                  'notSubjectToSection16', 'm_isDirector',
                                  'm_isOfficer', 'm_isTenPercentOwner',
                                  'm_isOther', 'transactionType',
                                  'transactionFormType', 'transactionCode',
                                  'equitySwapInvolved',
                                  'directOrIndirectOwnership',
                                  'natureOfOwnership'])],
                  verbose_feature_names_out=False)

In [22]:

def display_data_and_transform_columns(symbols):
  pl = conn.sql(f"SELECT * FROM lit_panel_aggr_v3 WHERE issuerTradingSymbol in ({symbols})").pl()
  display(pl)
  pl_encoded = transformer.fit_transform(pl)
  display(pl_encoded)



In [23]:
display_data_and_transform_columns("'GOOG', 'AAPL', 'TSLA', 'MSFT', 'FB', 'AMZN', 'TSM'")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

URL,acceptanceDatetime,accessionNumber,type,period,issuerCIK,documentType,periodOfReport,notSubjectToSection16,issuerTradingSymbol,m_isDirector,m_isOfficer,m_isTenPercentOwner,m_isOther,transactionType,securityTitle,transactionFormType,transactionCode,equitySwapInvolved,transactionTimeliness,directOrIndirectOwnership,natureOfOwnership,_total_transaction_shares,_total_transaction_amount,_minSharesOwnedFollowingTransaction,_maxSharesOwnedFollowingTransaction,_count
str,i64,str,str,i64,str,str,str,str,str,i64,i64,i64,i64,str,str,i64,str,str,str,str,str,f64,f64,i32,i32,i64
"""https://www.se…",20081027131200,"""0001127602-08-…","""4""",20081023,"""0001288776""","""4""","""2008-10-23""",null,"""GOOG""",1,0,0,0,"""nonDerivativeT…","""Class A Common…",4,"""S""","""0""",null,"""D""",null,140.0,48924.47,0,126,10
"""https://www.se…",20081125192747,"""0001127602-08-…","""4""",20081125,"""0001288776""","""4""","""2008-11-25""",null,"""GOOG""",1,0,0,0,"""nonDerivativeT…","""Class A Common…",4,"""S""","""0""",null,"""D""",null,140.0,38475.4888,0,126,10
"""https://www.se…",20081229151603,"""0001127602-08-…","""4""",20081223,"""0001288776""","""4""","""2008-12-23""",null,"""GOOG""",1,0,0,0,"""nonDerivativeT…","""Class A Common…",4,"""S""","""0""",null,"""D""",null,120.0,35998.428,0,108,10
"""https://www.se…",20081201152047,"""0001127602-08-…","""4""",20081126,"""0001288776""","""4""","""2008-11-26""",null,"""GOOG""",1,0,0,0,"""nonDerivativeT…","""Class A Common…",4,"""S""","""0""",null,"""D""",null,140.0,40332.6728,0,126,10
"""https://www.se…",20081105202324,"""0001127602-08-…","""4""",20081105,"""0001288776""","""4""","""2008-11-05""",null,"""GOOG""",0,1,0,0,"""nonDerivativeT…","""Class A Common…",4,"""S""","""0""",null,"""I""","""By Trust""",191.0,67843.5962,26750,26921,10
"""https://www.se…",20081104142328,"""0001127602-08-…","""4""",20081103,"""0001288776""","""4""","""2008-11-03""",null,"""GOOG""",0,1,0,0,"""nonDerivativeT…","""Class A Common…",4,"""S""","""0""",null,"""I""","""By Trust""",191.0,68212.0838,26750,26921,10
"""https://www.se…",20081105194857,"""0001127602-08-…","""4""",20081104,"""0001288776""","""4""","""2008-11-04""",null,"""GOOG""",0,1,0,0,"""nonDerivativeT…","""Class A Common…",4,"""S""","""0""",null,"""I""","""By Trust""",191.0,69382.9296,26750,26926,10
"""https://www.se…",20081202205839,"""0001127602-08-…","""4""",20081201,"""0001288776""","""4""","""2008-12-01""",null,"""GOOG""",0,1,0,0,"""nonDerivativeT…","""Class A Common…",4,"""S""","""0""",null,"""I""","""By Trust""",191.0,52314.3873,26750,26922,10
"""https://www.se…",20081110183846,"""0001127602-08-…","""4""",20081106,"""0001288776""","""4""","""2008-11-06""",null,"""GOOG""",0,1,0,0,"""nonDerivativeT…","""Class A Common…",4,"""S""","""0""",null,"""I""","""By Trust""",191.0,63487.2844,26750,26921,10


acceptanceDatetime,period,_total_transaction_shares,_total_transaction_amount,_minSharesOwnedFollowingTransaction,_maxSharesOwnedFollowingTransaction,_count,type_3,type_3/A,type_4,type_4/A,type_5,type_5/A,documentType_3,documentType_3/A,documentType_4,documentType_4/A,documentType_5,documentType_5/A,notSubjectToSection16_0,notSubjectToSection16_1,notSubjectToSection16_None,m_isDirector_0,m_isDirector_1,m_isOfficer_0,m_isOfficer_1,m_isTenPercentOwner_0,m_isTenPercentOwner_1,m_isOther_0,m_isOther_1,transactionType_derivativeHolding,transactionType_derivativeTransaction,transactionType_nonDerivativeHolding,transactionType_nonDerivativeTransaction,transactionType_None,transactionFormType_4.0,transactionFormType_5.0,…,natureOfOwnership_The Clover Irrevocable Nonexempt Trust U/A/D 6/27/2011,natureOfOwnership_The Clover Irrevocable Nonexempt Trust u/a/d 6/27/11,natureOfOwnership_The Erin Hoffmann 2017 Annuity Trust U/A/D 6/29/2017,natureOfOwnership_The Erin Hoffmann 2018 Annuity Trust U/A/D 11/15/2018,natureOfOwnership_The Erin Hoffmann 2018 Annuity Trust U/A/D/ 11/15/2018,natureOfOwnership_The HS Trust U/A/D 9/28/2011,natureOfOwnership_The Hampton 1999 Trust,natureOfOwnership_The Michael Schroepfer 2017 Annuity Trust U/A/D 6/29/2017,natureOfOwnership_The Michael Schroepfer 2018 Annuity Trust U/A/D 11/15/2018,natureOfOwnership_The Michael Schroepfer 2018 Annuity Trust U/A/D/ 11/15/2018,natureOfOwnership_Thomas Alton Burgum Trust,natureOfOwnership_Through synthetic index fund account,natureOfOwnership_Trust,natureOfOwnership_Trust 1,natureOfOwnership_Trust 3,natureOfOwnership_Trust 4,natureOfOwnership_Vallejo Ventures Trust,natureOfOwnership_Vijay Shriram TR UA 09/10/2021 2021 VS Irrevocable Trust,natureOfOwnership_by AJG Growth Fund,natureOfOwnership_by DFJ Fd IX Part LP,natureOfOwnership_by DFJ Fd X Part LP,"natureOfOwnership_by DFJ Fund IX, LP",natureOfOwnership_by DFJ Fund VIII LP,"natureOfOwnership_by DFJ Fund X, L.P.",natureOfOwnership_by DFJ Part IX LLC,natureOfOwnership_by DFJ Part VIII LLC,natureOfOwnership_by DFJ VIII Part LP,natureOfOwnership_by DFJGF 06 Prt LP,"natureOfOwnership_by DFJP GF 2006, LLC","natureOfOwnership_by DFJP X, LLC",natureOfOwnership_by GRAT,natureOfOwnership_by Seven Hills Trust,"natureOfOwnership_by TP Mgmt VIII, LLC",natureOfOwnership_by Trust,natureOfOwnership_by VEM II,natureOfOwnership_by VEP II,natureOfOwnership_None
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.785687,-0.78157,-0.051316,-0.011763,-0.134987,-0.135393,1.037711,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
-0.783959,-0.779774,-0.051316,-0.011774,-0.134987,-0.135393,1.037711,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
-0.782127,-0.778049,-0.05132,-0.011776,-0.134987,-0.135393,1.037711,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
-0.78262,-0.779757,-0.051316,-0.011772,-0.134987,-0.135393,1.037711,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,